### Simple COVID-19 stochastic model based on Anderson et al. (_Lancet 2020_)

Below I define a stochastic version of the model defined diagrammatically below taken from Anderson et al. (_Lancet 2020_), linked here: https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)30567-5/fulltext.

<img src="diagram-from-Anderson2020.png" width="600"/>

First the following imports are necessary...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Then the model parameters (all in units of 1/day) and initial compartment numbers must be defined...

In [2]:
lam, sig, gam1, gam2, gam3, alph, p = 1.0, 1.0/5.0, 1.0/5.0, 1.0/4.0, 1.0/4.0, 1.0/4.0, 0.1
S, E, I1, I2mild, I2severe, I3mild, I3severe, Y2, Y3, R = 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0

Finally, the number of realisations and the number of days to run must be chosen...

In [3]:
nreals = 100
ndays = 5

The model is coded in the following function...

In [26]:
# Package parameters and initial conditions all together
params = [lam, sig, gam1, gam2, gam3, alph, p]
initconds = [S*np.ones(nreals),E*np.ones(nreals), I1*np.ones(nreals), I2mild*np.ones(nreals),\
             I2severe*np.ones(nreals), I3mild*np.ones(nreals), I3severe*np.ones(nreals), Y2*np.ones(nreals),\
             Y3*np.ones(nreals), R*np.ones(nreals)]

# Define model function to output after specified time period
def covid_simple(initconds,params,reals,runtime,output_to_file): 
    # Open storage for output to file
    f = open(output_to_file, "w")
    # Unpack parameters and initial conditions
    [lam, sig, gam1, gam2, gam3, alph, p] = params
    [S, E, I1, I2mild, I2severe, I3mild, I3severe, Y2, Y3, R] = initconds 
    # Convert to float because I'm like that...
    runtime = float(runtime)   
    # Main loop
    t = np.zeros(reals)
    terminated = np.ones(reals)
    while np.any(t) < runtime:    
        # Terminate loop if everyone has recovered
        if np.all(S+E+I1+I2mild+I2severe+I3mild+I3severe+Y2+Y3) == 0: break
        # Update the event rates
        rate_S_E, rate_E_I1, rate_I1_I2, rate_I2severe_Y2 = lam*S, sig*E, gam1*I1, alph*I2severe
        rate_I2mild_I3mild, rate_I2severe_I3severe, rate_Y2_Y3 = gam2*I2mild, gam2*I2severe, gam2*Y2
        rate_I3mild_R, rate_I3severe_R, rate_Y3_R = gam3*I3mild, gam3*I3severe, gam3*Y3
        # Sum the rates
        rate_sum = rate_S_E + rate_E_I1 + rate_I1_I2 + rate_I2severe_Y2 + rate_I2mild_I3mild + \
                   rate_I2severe_I3severe + rate_Y2_Y3 + rate_I3mild_R + rate_I3severe_R + rate_Y3_R
        # Avoid pathologies at the end
        rate_sum[rate_sum<=0.0] = 1.0
        # Take step forward in time and determine when to stop each run
        t = t + terminated*np.random.exponential(1.0/rate_sum,size=reals) 
        terminated = (t<runtime)
        # Choose the event to occur in each realisation
        severe = np.random.binomial(1,p,size=reals)
        event = np.random.uniform(size=reals)
        event_probs = np.asarray([rate_S_E/rate_sum,rate_E_I1/rate_sum,\
                                  rate_I1_I2/rate_sum,rate_I2severe_Y2/rate_sum,rate_I2mild_I3mild/rate_sum,\
                                  rate_I2severe_I3severe/rate_sum,rate_Y2_Y3/rate_sum,rate_I3mild_R/rate_sum,\
                                  rate_I3severe_R/rate_sum,rate_Y3_R/rate_sum])
        cumul_probs = np.cumsum(event_probs,axis=0)
        S = S - (event<cumul_probs[0])
        E = E + (event<cumul_probs[0]) - (cumul_probs[1]>event)*(event>=cumul_probs[0])
        I1 = I1 + (cumul_probs[1]>event)*(event>=cumul_probs[0]) - (cumul_probs[2]>event)*(event>=cumul_probs[1])
        I2mild = I2mild + (severe==0)*(cumul_probs[2]>event)*(event>=cumul_probs[1]) \
                        - (cumul_probs[4]>event)*(event>=cumul_probs[3])
        I2severe = I2mild + (severe==1)*(cumul_probs[2]>event)*(event>=cumul_probs[1]) \
                          - (cumul_probs[3]>event)*(event>=cumul_probs[2]) \
                          - (cumul_probs[5]>event)*(event>=cumul_probs[4])
        I3mild = I3mild + (cumul_probs[4]>event)*(event>=cumul_probs[3]) \
                        - (cumul_probs[7]>event)*(event>=cumul_probs[6])
        I3severe = I3severe + (cumul_probs[5]>event)*(event>=cumul_probs[4]) \
                            - (cumul_probs[8]>event)*(event>=cumul_probs[7])
        Y2 = Y2 + (cumul_probs[3]>event)*(event>=cumul_probs[2]) - (cumul_probs[6]>event)*(event>=cumul_probs[5])
        Y3 = Y3 + (cumul_probs[6]>event)*(event>=cumul_probs[5]) - (cumul_probs[9]>event)*(event>=cumul_probs[8])
        R = R + (cumul_probs[7]>event)*(event>=cumul_probs[6]) + (cumul_probs[8]>event)*(event>=cumul_probs[7]) \
                                                               + (cumul_probs[9]>event)*(event>=cumul_probs[8])
        # Store values in file
        f.write(','.join(map(str,np.concatenate([t, S, E, I1, I2mild, I2severe, \
                                                 I3mild, I3severe, Y2, Y3, R]).ravel().tolist()))+'\n')
        f.flush()
    # Close file
    f.close()
    # Setup output column name list
    column_names = []
    for l in ['t', 'S', 'E', 'I1', 'I2mild', 'I2severe', 'I3mild', 'I3severe', 'Y2', 'Y3', 'R']:
        for r in range(0,nreals):
            column_names.append(l+'_'+str(r))
    # Load dataframe with column names
    df = pd.read_csv(output_to_file,names=column_names)
    # Output resulting compartment numbers to a pandas dataframe
    return df

In [27]:
df = covid_simple(initconds,params,nreals,ndays,'plot_data/test.csv')

In [28]:
df

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,R_90,R_91,R_92,R_93,R_94,R_95,R_96,R_97,R_98,R_99
0,0.000550,0.001305,0.000740,0.000103,0.001926,0.000562,0.000747,0.003424,0.000775,0.000480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000776,0.002499,0.000812,0.001495,0.002596,0.000931,0.003088,0.003467,0.002798,0.000589,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001173,0.002806,0.001391,0.001622,0.002633,0.001180,0.003546,0.003557,0.003885,0.001280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001476,0.003000,0.002905,0.002865,0.002920,0.002400,0.003985,0.005503,0.004088,0.005194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.005244,0.005292,0.003690,0.003200,0.003626,0.002897,0.004042,0.005613,0.004531,0.005756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8830,5.004513,5.000338,5.001687,5.000347,5.002443,5.000126,5.006776,5.002217,5.000384,5.002498,...,2475.0,2374.0,2416.0,2468.0,2427.0,2411.0,2426.0,2440.0,2455.0,2382.0
8831,5.004513,5.000338,5.001687,5.000347,5.002443,5.000126,5.006776,5.002217,5.000384,5.002498,...,2475.0,2374.0,2417.0,2469.0,2427.0,2412.0,2426.0,2441.0,2455.0,2383.0
8832,5.004513,5.000338,5.001687,5.000347,5.002443,5.000126,5.006776,5.002217,5.000384,5.002498,...,2476.0,2374.0,2417.0,2470.0,2427.0,2412.0,2426.0,2441.0,2456.0,2384.0
8833,5.004513,5.000338,5.001687,5.000347,5.002443,5.000126,5.006776,5.002217,5.000384,5.002498,...,2476.0,2375.0,2418.0,2471.0,2428.0,2413.0,2426.0,2441.0,2457.0,2384.0
